In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import random

import torchvision
from PIL import Image
from torch.utils.data import DataLoader,Dataset,random_split

from torchvision import transforms
from torchvision.datasets import CocoDetection

In [2]:
# 设置随机数种子
def set_seed(seed):
    torch.manual_seed(seed)  # 设置PyTorch的随机数种子
    np.random.seed(seed)     # 设置Numpy的随机数种子
    random.seed(seed)       # 设置Python内置随机数生成器的种子
    # 如果使用CUDA
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # 如果使用多GPU
        # 以下两个设置可以帮助保持CUDA生成的随机数在每次运行时保持一致
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# 使用
seed = 42  # 设置随机数种子
set_seed(seed)

In [3]:
from models.Unet import Unet
from data.UnetDataset import ImgData
from torch.utils.data import Dataset, DataLoader,random_split

In [4]:
img_path=r"E:\developer\A-Study\Image_Segmentation\UNET-Car\Dataset\imgs\train"
mask_path=r"E:\developer\A-Study\Image_Segmentation\UNET-Car\Dataset\masks\train"

trans1 = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor()
])

In [5]:
dataset=ImgData(img_path,mask_path,trans1)
train,val=random_split(dataset,[0.9,0.1])
train_loader = DataLoader(train,batch_size=8,shuffle=True)

In [6]:
from models.BiSeNet import BiSeNet
model = BiSeNet(num_classes=1)
model=model.to('cuda')
Epoch=50
LR=1e-4
criterion=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=LR)


def get_IOU(SR,GT,threshold=0.5):
    SR = SR > threshold
    GT = GT == torch.max(GT)
    # 计算交集和并集
    intersection = (SR * GT).sum()
    union = SR.sum() + GT.sum() - intersection
    
    # 避免除以零
    iou = intersection / (union + 1e-6)
    
    return iou

In [7]:
save_path=r"F:\items\Seg\SegInOne\model_zoo\BiSeNet.pth"
best_iou=0.0
for epoch in range(Epoch):
    
    model.train()
    for i,data in enumerate(train_loader):
        img,mask=data
        output=model(img.to('cuda'))
        loss=criterion(output,mask.to('cuda'))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss)
    
    model.eval()
    total_iou=0.0
    with torch.no_grad():
        for data in val:
            img,mask=data
            img=img.to('cuda')
            mask=mask.to('cuda')
            output=model(img.unsqueeze(0))
            iou=get_IOU(SR=output.squeeze(0),GT=mask)
            total_iou+=iou
        avg_iou=total_iou/len(val)
        if avg_iou>best_iou:
            best_iou=avg_iou
            print("new best model")
            torch.save(model.state_dict(),save_path)
        

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[8, 128, 28, 28] to have 3 channels, but got 128 channels instead